<a href="https://colab.research.google.com/github/jx-dohwan/Aiffel_EGLMS_Project/blob/main/%5BExp_10%5DLet's_make_a_translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Exp_10]Let's make a translator

## 1.import 및 데이터로드

### 1) Import

In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

### 2) 데이터로드

In [6]:
file_path = "/content/drive/MyDrive/인공지능/아이펠/ES/data/translator_seq2seq/fra.txt"
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플수:',len(lines))
lines.sample(5)

전체 샘플수: 194513


,eng,fra,cc
66531,Tom changed the subject.,Tom a changé de sujet.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
193434,Spending time with your significant other shou...,Passer du temps avec ta bien-aimée devrait êtr...,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
104080,The ship set sail for Bombay.,Le navire fit voile pour Bombay.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
31562,That's my daughter.,C'est ma fille.,CC-BY 2.0 (France) Attribution: tatoeba.org #7...
170268,Don't feel embarrassed. These things happen.,Ne vous sentez pas gêné. Ces choses arrivent.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...



데이터에서 상위 33,000개의 샘플만 사용해주세요.

33000개 중 3000개는 테스트 데이터로 분리하여 모델을 학습한 후에 번역을 테스트 하는 용도로 사용합니다

In [3]:
lines = lines[['eng', 'fra']][:33000]
lines.sample(5)

,eng,fra
How big is he?,Il est grand comment ?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
Are you smiling?,Êtes-vous en train de sourire ?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
Tom left it.,Tom l'a laissé.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
You may refuse.,Vous pouvez refuser.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
This seat's free.,Cette place est libre.,CC-BY 2.0 (France) Attribution: tatoeba.org #8...


## 2. 정제, 정규화, 전처리
- 구두점(Punctuation)을 단어와 분리
- 소문자로 change
- 띄어쓰기 단위로 토큰을 수행

- https://soraji.github.io/python/2019/07/21/rex/
- https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer

In [ ]:
import spacy

def preprocess_sentence(sentence):
  sentence = sentence.str.lower()
  sentence = sentence.str.replace(pat=r'([?.!,¿])', repl =r" \1 ")
  sentence = sentence.str.replace(pat=r'[" "]+', repl =" ")
  sentence = sentence.str.replace(pat=r"[^a-zA-Z?.!,¿]+", repl =" ")
  sentence = sentence.str.strip() 

  return sentence

In [101]:
input_text = preprocess_sentence(lines.eng)
eng_tokenizer = Tokenizer(split=' ', filters="")   # 문자 단위로 Tokenizer를 생성합니다. 
eng_tokenizer.fit_on_texts(input_text)               # 50000개의 행을 가진 eng의 각 행에 토큰화를 수행
input_text = eng_tokenizer.texts_to_sequences(input_text)    # 단어를 숫자값 인덱스로 변환하여 저장
input_text[:3]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys


[[47, 1], [47, 1], [47, 1]]

In [102]:
input_text = preprocess_sentence(lines.fra)
fra_tokenizer = Tokenizer(split=' ', filters="")   # 문자 단위로 Tokenizer를 생성합니다. 
fra_tokenizer.fit_on_texts(input_text)               # 50000개의 행을 가진 eng의 각 행에 토큰화를 수행
input_text = fra_tokenizer.texts_to_sequences(input_text)    # 단어를 숫자값 인덱스로 변환하여 저장
input_text[:3]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys


[[116, 42], [814, 1], [2397, 42]]

## 3. 디코더의 문장에 시작 토큰과 종료 토큰 입력

## 4. 케라스의 토크나이저로 텍스트를 숫자로 바꿔보세요
- https://wikidocs.net/31766

## 5. 임베딩 층(Embedding layer) 사용하기
- https://wikidocs.net/33793
- 주의할 점 : 인코더와 디코더의 임베딩 층은 서로 다른 임베딩 층을 사용해야 하지만 디코더의 훈련 과정과 테스트 과정에서의 임베딩층은 동일해야 한다.

## 6. 모델 구현하기

## 7. 모델 평가하기

## 8. 회고

### 시행착오

In [67]:
import re
def preprocess_sentence(sentence):
    sentence = sentence.lower() # 1 소문자로 바꾸고
    sentence = re.sub(r"([?.!,¿])", r" \1 ", str(sentence)) # 2 특수문자 양쪽에 공백을 넣는다.
    sentence = re.sub(r'[" "]+', " ", str(sentence)) # 3 여러개의 공백은 하나의 공백으로 바꾼다.
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", str(sentence)) # 4 a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꾼다. 
    tokenizer  = Tokenizer(split=' ')  
    tokenizer .fit_on_texts(sentence)               # 50000개의 행을 가진 eng의 각 행에 토큰화를 수행
    sentence = tokenizer.texts_to_sequences(sentence) 
    return sentence

In [68]:
re.sub(r'[" "]+', ' ',(str(re.sub(r'([?.!,])', r' \1 ', str(str(lines.eng).lower())))))

"0 go . \n1 go . \n2 go . \n3 hi . \n4 hi . \n . . . \n194508 a carbon footprint is the amount of carbon dio . . . \n194509 death is something that we're often discourage . . . \n194510 since there are usually multiple websites on a . . . \n194511 if someone who doesn't know your background sa . . . \n194512 it may be impossible to get a completely error . . . \nname: eng , length: 194513 , dtype: object"

In [69]:
input_text = preprocess_sentence(lines.eng)
input_text[:3]


AttributeError: ignored

In [ ]:
lines.eng[:3]

0    Go.
1    Go.
2    Go.
Name: eng, dtype: object

In [ ]:
input_text[:3]

[[], [9], [2]]

### 시행착오

In [ ]:
import spacy

def preprocess_sentence(sentence):
  sentence = sentence.str.lower()
  sentence = sentence.str.replace(pat=r'([?.!,¿])', repl =r" \1 ")
  sentence = sentence.str.replace(pat=r'[" "]+', repl =" ")
  sentence = sentence.str.replace(pat=r"[^a-zA-Z?.!,¿]+", repl =" ")
  sentence = sentence.str.strip() 

  return sentence
  

In [ ]:
fra = preprocess_sentence(lines.fra)
fra.str.split()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys


0                                                   [va, !]
1                                               [marche, .]
2                                                [bouge, !]
3                                                [salut, !]
4                                                [salut, .]
                                ...                        
194508    [une, empreinte, carbone, est, la, somme, de, ...
194509    [la, mort, est, une, chose, qu, on, nous, d, c...
194510    [puisqu, il, y, a, de, multiples, sites, web, ...
194511    [si, quelqu, un, qui, ne, conna, t, pas, vos, ...
194512    [il, est, peut, tre, impossible, d, obtenir, u...
Name: fra, Length: 194513, dtype: object

In [ ]:
[salut, .]

In [ ]:
len(set[0])
set[0][0]

'g'

https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer

In [ ]:
tokenizer  = Tokenizer(split=' ',filters='')  
tokenizer .fit_on_texts(set)               # 50000개의 행을 가진 eng의 각 행에 토큰화를 수행
sentence = tokenizer.texts_to_sequences(set) 

In [ ]:
sentence

[[47, 1],
 [47, 1],
 [47, 1],
 [2374, 1],
 [2374, 1],
 [409, 109],
 [409, 109],
 [409, 109],
 [409, 109],
 [409, 109],
 [409, 109],
 [409, 109],
 [409, 109],
 [409, 1],
 [409, 1],
 [409, 1],
 [409, 1],
 [409, 1],
 [409, 1],
 [409, 1],
 [409, 1],
 [76, 6],
 [3441, 109],
 [3441, 109],
 [3441, 109],
 [4089, 109],
 [4089, 109],
 [4089, 109],
 [442, 109],
 [88, 109],
 [1092, 1],
 [1092, 1],
 [1334, 109],
 [1334, 1],
 [179, 109],
 [179, 109],
 [179, 109],
 [258, 109],
 [258, 109],
 [258, 109],
 [258, 1],
 [258, 1],
 [258, 1],
 [258, 1],
 [925, 1],
 [925, 1],
 [47, 39, 1],
 [47, 39, 1],
 [47, 39, 1],
 [1614, 109],
 [1614, 109],
 [2, 86, 1],
 [2, 86, 1],
 [2, 196, 1],
 [2, 143, 109],
 [2, 143, 109],
 [2, 143, 1],
 [2008, 61, 109],
 [1306, 1],
 [1306, 1],
 [1306, 1],
 [1306, 1],
 [1306, 1],
 [1306, 1],
 [1306, 1],
 [1306, 1],
 [1306, 1],
 [1306, 1],
 [1306, 1],
 [1306, 1],
 [1069, 1],
 [1069, 1],
 [1069, 1],
 [206, 6],
 [1561, 109],
 [1561, 109],
 [1561, 109],
 [193, 12, 1],
 [193, 12, 1],
 [19

0                                                       go.
1                                                       go.
2                                                       go.
3                                                       hi.
4                                                       hi.
                                ...                        
194508    a carbon footprint is the amount of carbon dio...
194509    death is something that we're often discourage...
194510    since there are usually multiple websites on a...
194511    if someone who doesn't know your background sa...
194512    it may be impossible to get a completely error...
Name: eng, Length: 194513, dtype: object

In [ ]:
import re
def preprocess_sentence(sentence):
    sentence = str.extract.lower() # 1 소문자로 바꾸고
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2 특수문자 양쪽에 공백을 넣는다.
    sentence = re.sub(r'[" "]+', " ", sentence) # 3 여러개의 공백은 하나의 공백으로 바꾼다.
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4 a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꾼다. 
    tokenizer  = Tokenizer(split=' ')  
    tokenizer .fit_on_texts(sentence)               # 50000개의 행을 가진 eng의 각 행에 토큰화를 수행
    sentence = tokenizer.texts_to_sequences(sentence) 
    return sentence

In [ ]:
set = lines.eng.str.lower()
set = set.str,extract("([?.!,¿])", r" \1 ")
set

NameError: ignored

In [ ]:
print(str(lines.eng).lower())
str(lines.eng).lower()[:3]
lines.eng[:3]

0                                                       go.
1                                                       go.
2                                                       go.
3                                                       hi.
4                                                       hi.
                                ...                        
194508    a carbon footprint is the amount of carbon dio...
194509    death is something that we're often discourage...
194510    since there are usually multiple websites on a...
194511    if someone who doesn't know your background sa...
194512    it may be impossible to get a completely error...
name: eng, length: 194513, dtype: object


0    Go.
1    Go.
2    Go.
Name: eng, dtype: object

In [ ]:
re.sub(r'[" "]+', ' ',(str(re.sub(r'([?.!,])', r' \1 ', str(str(lines.eng).lower())))))

"0 go . \n1 go . \n2 go . \n3 hi . \n4 hi . \n . . . \n194508 a carbon footprint is the amount of carbon dio . . . \n194509 death is something that we're often discourage . . . \n194510 since there are usually multiple websites on a . . . \n194511 if someone who doesn't know your background sa . . . \n194512 it may be impossible to get a completely error . . . \nname: eng , length: 194513 , dtype: object"

In [ ]:

input_text = preprocess_sentence(lines['eng'])
input_text[:3]


AttributeError: ignored

In [ ]:
lines.eng[:3]

0    Go.
1    Go.
2    Go.
Name: eng, dtype: object

In [ ]:
input_text[:3]

[[], [9], [2]]

- https://acdongpgm.tistory.com/166

In [ ]:
import re
def preprocess_sentence(sentence):
  df["title"] = df["title"].str.replace(pat=r'[^\w]', repl=r'', regex=True)
    sentence = sentence.str.extract.lower() # 1 소문자로 바꾸고
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2 특수문자 양쪽에 공백을 넣는다.
    sentence = re.sub(r'[" "]+', " ", sentence) # 3 여러개의 공백은 하나의 공백으로 바꾼다.
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4 a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꾼다. 
    tokenizer  = Tokenizer(split=' ')  
    tokenizer .fit_on_texts(sentence)               # 50000개의 행을 가진 eng의 각 행에 토큰화를 수행
    sentence = tokenizer.texts_to_sequences(sentence) 
    return sentence